# 11 Sklearn Features I Sincerely Hope You Learn After Using It For a Year
## TODO
![](images/pexels.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@psco?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pineapple Supply Co.</a>
        on 
        <a href='https://www.pexels.com/photo/photo-of-three-pineapples-surrounded-by-balloons-1071882/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

# Setup

In [1]:
import logging
import time
import warnings

import catboost as cb
import datatable as dt
import joblib
import lightgbm as lgbm
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import xgboost as xgb
from optuna.samplers import TPESampler
from sklearn.compose import (
    ColumnTransformer,
    make_column_selector,
    make_column_transformer,
)
from sklearn.impute import SimpleImputer
from sklearn.metrics import log_loss, mean_squared_error
from sklearn.model_selection import (
    KFold,
    StratifiedKFold,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

logging.basicConfig(
    format="%(asctime)s - %(message)s", datefmt="%d-%b-%y %H:%M:%S", level=logging.INFO
)
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings("ignore")
pd.set_option("float_format", "{:.5f}".format)

In [4]:
from sklearn.datasets import *

X, y = make_regression(n_samples=10000, n_features=10)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, random_state=1121218, test_size=0.2
)

# Introduction

# 1. FunctionTransformer

# 2. BaseEstimator and TransformerMixin

# 3. TransformedTargetRegressor

# 4. QuadraticDiscriminantAnalysis

# 5. Voting Classifier/Regresssor

# 6. Stacking Classifier/Regressor

# 7. IsolatioForest

# 8. LocalOutlierFactor

# 9. QuantileTransformer

# 10. PCA + tSNE

# 11. HTML Estimator Representation

# Summary